# Open-Source only RAG for Testing Scenario Creation

- Sentence Transformers as embedding model
- Postgres as vector store
- Llama.cpp as LLM

#### Sentence Transformers

In [1]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached striprtf-0.0.26-py3-none-any.whl.metadata (2.1 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.36-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached pillow-11.0.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached tiktoken-0.8.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 

In [2]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/Users/busraoguzoglu/Desktop/Okul/Dersler/test_verification/project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Llama CPP

In [3]:
%pip install llama-cpp-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/busraoguzoglu/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         

#### Postgres

In [5]:
%pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

Note: you may need to restart the kernel to use updated packages.


In [6]:
import psycopg2

# Define your database connection parameters
db_name = "rag_db"  # Use the existing database name here
host = "localhost"
password = "password"  # Replace with your actual PostgreSQL password
port = "5432"  # Default PostgreSQL port
user = "myuser"  # Replace with your actual PostgreSQL username

# Connect directly to the 'rag_db' database
conn = psycopg2.connect(
    dbname=db_name,  # Connect directly to 'rag_db'
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

# Optionally, perform any operations on 'rag_db' using a cursor
with conn.cursor() as c:
    # Drop and create operations are not needed if the database already exists
    # Here you can perform other database setup actions if necessary
    print(f"Connected to {db_name} successfully.")

# Close the connection when done
#conn.close()

Connected to rag_db successfully.


Create table (checks if exist, truncates if already exist and generates again)

In [7]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
from llama_index.vector_stores.postgres import PGVectorStore

# Connection parameters
db_params = {
    "user": "myuser",      # Replace with your actual username
    "password": "password",  # Replace with your actual password
    "host": "localhost",   # Adjust if your database is hosted elsewhere
    "port": "5432",        # Default PostgreSQL port
    "database": "rag_db"   # Replace with your actual database name
}

# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

# Update table name to include 'data_' prefix
table_name = "data_software_test_table"  # Postgre adds 'data' in front of table_name
embed_dim = 384  # Embedding dimension

try:
    with engine.connect() as conn:
        # Check if the table exists
        result = conn.execute(text(f"""
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE table_schema = 'public' AND table_name = :table_name
            );
        """), {"table_name": table_name}).scalar()

        if result:
            print(f"Table '{table_name}' exists. Cleaning it...")
            conn.execute(text(f"TRUNCATE TABLE {table_name};"))
        else:
            print(f"Table '{table_name}' does not exist. Creating it...")

        # Create a new table for vector store
        vector_store = PGVectorStore.from_params(
            database=db_params["database"],
            host=db_params["host"],
            password=db_params["password"],
            port=db_params["port"],
            user=db_params["user"],
            table_name="pmc_table",  # Use unprefixed name; PGVectorStore adds 'data_' automatically
            embed_dim=embed_dim,
        )
        print(f"Table '{table_name}' has been created.")
except OperationalError as e:
    print(f"Error: {e}")
    print("Make sure the database exists and connection parameters are correct.")

Table 'data_software_test_table' does not exist. Creating it...
Table 'data_software_test_table' has been created.


## Ingestion Pipeline

### Data Loading and Preprocessing:

Preprocessing of the document does this:

1- Remove all text after the stopwords, like 'References', because they do not have any information for us.

2- Remove section numbering, letters, extra spaces, new lines.

3- Remove 'tables' from the text.

In [8]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter

## Preprocessing 

### HTML Preprocessing

In [25]:
from bs4 import BeautifulSoup
import re
from pathlib import Path

def extract_text_and_elements(html_file_path):
    """Extract meaningful information from static HTML."""
    # Load the HTML content from the file
    with open(html_file_path, "r", encoding="utf-8") as file:
        html_content = file.read()
    
    # Parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Extract the title of the page
    title = soup.title.string.strip() if soup.title else "No Title"
    
    # Extract meta description (if available)
    meta_description = ""
    meta_tag = soup.find("meta", attrs={"name": "description"})
    if meta_tag and meta_tag.get("content"):
        meta_description = meta_tag["content"]
    
    # Extract all visible text (paragraphs, headings, etc.)
    text_content = []
    for tag in soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6"]):  # Common text tags
        text = tag.get_text(strip=True)
        if text:
            text_content.append(text)
    
    # Extract all button labels
    buttons = [button.get_text(strip=True) for button in soup.find_all("button")]
    
    # Extract all links (text and URLs)
    links = []
    for link in soup.find_all("a", href=True):
        link_text = link.get_text(strip=True)
        links.append({"text": link_text, "url": link["href"]})
    
    # Structure the extracted information
    extracted_content = {
        "title": title,
        "meta_description": meta_description,
        "text_content": "\n".join(text_content),
        "buttons": buttons,
        "links": links,
    }
    
    return extracted_content

def preprocess_for_rag(content):
    """Preprocess extracted HTML content for use in a RAG system."""
    # Combine title, meta description, text content, and button labels
    combined_content = (
        f"Title: {content['title']}\n\n"
        f"Meta Description: {content['meta_description']}\n\n"
        f"Text Content:\n{content['text_content']}\n\n"
        f"Buttons:\n{', '.join(content['buttons'])}\n\n"
        f"Links:\n"
    )
    # Add links in a readable format
    for link in content["links"]:
        combined_content += f"- {link['text']} ({link['url']})\n"
    
    return combined_content

# Input and Output Paths
html_file_path = Path("./data/imdbhtml.html")  # Replace with your actual HTML file path
output_path = Path("./processed_html.txt")  # File to save processed output

# Extract and preprocess the HTML content
extracted_content = extract_text_and_elements(html_file_path)
processed_content = preprocess_for_rag(extracted_content)

# Write the processed content into an output file
with output_path.open("w", encoding="utf-8") as output_file:
    output_file.write(processed_content)

print(f"Processed HTML content written to {output_path}")

Processed HTML content written to processed_html.txt


### pdf and txt preprocessing

In [26]:
import re
from pathlib import Path
from bs4 import BeautifulSoup  # For HTML parsing
from llama_index.readers.file import PyMuPDFReader

# Define stop keywords
stop_keywords = []

def preprocess_text(text, stop_keywords):
    """Preprocess text by stopping at the earliest occurrence of any keyword, ignoring case."""
    earliest_position = len(text)  # Default to the end of the text
    
    for keyword in stop_keywords:
        pattern = rf"\b{re.escape(keyword)}\b"  # Match exact word boundaries
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            print(f"Keyword '{keyword}' found at position {match.start()}")  # Debugging
            earliest_position = min(earliest_position, match.start())
    
    # Truncate the text at the earliest keyword match
    return text[:earliest_position]

def clean_text(block_text):
    """Clean and process individual blocks of text."""
    block_text = " ".join(block_text.split())  # Remove extra spaces and newlines
    block_text = re.sub(r'^\d+\.\s+', '', block_text)  # Remove section numbering
    block_text = re.sub(r'[^A-Za-z0-9\s,.]', '', block_text)  # Keep only basic punctuation and letters

    # Heuristic: Skip blocks that appear to be tables (e.g., many numbers or special formats)
    if len(block_text) < 10 or re.match(r'Page \d+', block_text):
        return None  # Skip short blocks or page numbers

    # Remove blocks that have a high density of numbers or specific patterns (tables)
    if re.search(r'\d{2,}', block_text) and re.search(r'(\d+\.\d+|\d+%)', block_text):
        return None  # Skip blocks with many numeric values

    # Remove blocks with common table headers like 'Table', 'Mediation Effect', etc.
    if re.search(r'\b(Table|Effect|Mediation|Summary|IV|Mediator|Cont)\b', block_text, re.IGNORECASE):
        return None

    return block_text

In [27]:
# Paths to input files
pdf_file_path = Path("./data/IMDb_Software_Design_Document.pdf")  # Replace with your actual PDF file path
html_file_path = Path("./data/processed_html.txt")  # Replace with your actual HTML file path
output_path = Path("./processed_documents.txt")  # File to save processed documents

In [28]:
documents = []

# Process the PDF file
loader = PyMuPDFReader()
loaded_docs = loader.load(file_path=str(pdf_file_path))
print(f"{pdf_file_path.name}: {len(loaded_docs)} documents loaded")

for doc_idx, doc in enumerate(loaded_docs):
    processed_text = preprocess_text(doc.text, stop_keywords)  # Use `doc.page_content` for text
    
    cleaned_blocks = []
    for block in processed_text.split("\n"):
        cleaned_block = clean_text(block)
        if cleaned_block:
            cleaned_blocks.append(cleaned_block)
    
    final_text = "\n".join(cleaned_blocks)
    
    # Append a dictionary to the `documents` list
    documents.append({
        "source": pdf_file_path.name,
        "content": final_text
    })

# Process the plain text file
with open(html_file_path, "r", encoding="utf-8") as txt_file:
    txt_content = txt_file.read()

# Preprocess the text
processed_text = preprocess_text(txt_content, stop_keywords)

cleaned_blocks = []
for block in processed_text.split("\n"):
    cleaned_block = clean_text(block)
    if cleaned_block:
        cleaned_blocks.append(cleaned_block)

final_text = "\n".join(cleaned_blocks)

# Append the processed text file to the documents list
documents.append({
    "source": html_file_path.name,
    "content": final_text
})

# Write the processed text into the output file
with output_path.open("w", encoding="utf-8") as output_file:
    for idx, doc in enumerate(documents):
        output_file.write(f"Document {idx} from {doc['source']}:\n")
        output_file.write(doc['content'] + "\n")
        output_file.write("="*80 + "\n")  # Separator between documents

print(f"Processed documents written to {output_path}")

IMDb_Software_Design_Document.pdf: 8 documents loaded
Processed documents written to processed_documents.txt


In [29]:
print(documents[2])
print(len(documents))

{'source': 'IMDb_Software_Design_Document.pdf', 'content': ' 1.4 References\n IMDb official website httpswww.imdb.com\n REST API design principles\n OWASP security guidelines\n 2. System Overview\nThe IMDb web platform is a threetiered architecture consisting of\nFrontend A responsive web interface for user interaction.\nBackend A scalable service layer handling business logic and APIs.\nDatabase A relational and nonrelational hybrid system for storing structured and\nunstructured data.\n 3. Requirements\n 3.1 Functional Requirements\nSearch Functionality Provide robust search with autosuggestions for movies, TV shows, and\nDetailed Pages Display detailed information for titles, actors, and crew.\nUser Accounts Enable user registration, login, and profile management.\nWatchlist Allow users to add titles to personal watchlists.'}
9


### Splitting Documents

In [30]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [33]:
text_chunks = []
# Maintain relationship with source doc index to help inject doc metadata
doc_idxs = []

for doc_idx, doc in enumerate(documents):
    # Use dictionary-style indexing to access 'content'
    cur_text_chunks = text_parser.split_text(doc['content'])
    
    # Optionally print chunk info for debugging
    # print(f"Document {doc_idx} has {len(cur_text_chunks)} chunks")
    
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))  # Maintain mapping between chunks and their original document

In [34]:
print(len(text_chunks))

12


### Construct Nodes from Text Chunks

In [36]:
from llama_index.core.schema import TextNode

nodes = []

for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
        metadata={
            "source": documents[doc_idxs[idx]]["source"],  # Access 'source' from the dictionary
            "doc_index": doc_idxs[idx]  # Optional: document index
        }
    )
    nodes.append(node)

In [37]:
nodes[1]

TextNode(id_='b600e2bd-2f5a-49b6-9d5e-d25d2e420401', embedding=None, metadata={'source': 'IMDb_Software_Design_Document.pdf', 'doc_index': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Security Considerations\nTesting and Validation\nDeployment Strategy\nFuture Enhancements\n 1. Introduction\n 1.1 Purpose\nThe purpose of this document is to outline the software design for the IMDb web platform. IMDb is a\nwidelyused online database that provides information about movies, TV shows, actors, and\nproduction crew. This document serves as a blueprint for the development, maintenance, and\nscalability of the platform.\n 1.2 Scope\nThe IMDb web platform allows users to\n Search for and browse movies, TV shows, and celebrity profiles.\n View detailed information about titles, including cast, crew, reviews, and ratings.\n Create user accounts to rate and review titles, create watchlists, a

### Generate Embeddings for each Node

Using sentence_transformers

In [38]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="text")
    )
    node.embedding = node_embedding

In [39]:
print(type(nodes[0].embedding))
print(nodes[0].embedding)
print(len(nodes))

<class 'list'>
[-0.07680513709783554, -0.03635895252227783, 0.00535182049497962, -0.05403197184205055, 0.01925588957965374, 0.0034800402354449034, -0.041778065264225006, 0.012374467216432095, -0.014524527825415134, -0.007508639711886644, 0.04225458577275276, 0.002809097059071064, 0.02496599219739437, -0.0010692260693758726, -0.011186065152287483, 0.02664264850318432, 0.01867816224694252, -0.023114057257771492, -0.007859906181693077, 0.04352802038192749, 0.04277212545275688, -0.021671568974852562, -0.013970378786325455, -0.034672338515520096, 0.012791698798537254, 0.05684015899896622, -0.018318742513656616, -0.034304335713386536, -0.04022960737347603, -0.18466687202453613, -0.005947024095803499, -0.019826244562864304, 0.030649829655885696, 0.028392869979143143, -0.013961427845060825, -0.012469947338104248, 0.003333654487505555, 0.024638045579195023, -0.04771852120757103, -0.01206614263355732, -6.635607860516757e-05, 0.015827082097530365, 0.007107449695467949, -0.06058720871806145, 0.012

### Load Nodes into a Vector Store

We now insert these nodes into our `PostgresVectorStore`.

In [40]:
vector_store.add(nodes)

['831648af-d420-4ae4-8ad0-96d6edb16642',
 'b600e2bd-2f5a-49b6-9d5e-d25d2e420401',
 'e4b67ed6-51e4-4d13-a1b3-e02389f393c1',
 '14ffb16e-0225-4f24-ad87-406b5ac29c53',
 '05560ba7-5b59-4d38-b655-51b70ecdae2c',
 '6914463a-ae1c-43d7-b1cf-d97320995026',
 'e3afa880-02b9-4bde-ba7f-45c673307e0c',
 'da3deb06-2d9b-429f-b5ca-534353abca14',
 'ae042cad-2114-4626-ac75-faecc6014ed8',
 'c001fa4c-2025-48d6-afe9-64ef00d40f86',
 '105eecc8-ef55-4f2b-a955-3eb3804d95e8',
 'ce6ed192-cfda-4d72-9875-2f2625bad722']

## Retrieval Pipeline

In [41]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List
from llama_index.core.schema import NodeWithScore
from typing import Optional
from llama_index.core.vector_stores import VectorStoreQuery

In [42]:
class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [43]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

retriever

## Response

In [44]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [53]:
query_str = "What can be a test scenario in IMDB website about login page?"

response = query_engine.query(query_str)

Llama.generate: 29 prefix-match hit, remaining 315 prompt tokens to eval

llama_print_timings:        load time =   11934.30 ms
llama_print_timings:      sample time =       9.63 ms /   256 runs   (    0.04 ms per token, 26594.64 tokens per second)
llama_print_timings: prompt eval time =    6596.07 ms /   315 tokens (   20.94 ms per token,    47.76 tokens per second)
llama_print_timings:        eval time =   31861.58 ms /   255 runs   (  124.95 ms per token,     8.00 tokens per second)
llama_print_timings:       total time =   38608.65 ms /   570 tokens


In [54]:
print(str(response))



Possible test scenarios for the login page of the IMDB website include:

1. Successful login: Test that a valid user can successfully log in to their account using their email address and password.

2. Incorrect login credentials: Test that an invalid email address or password results in an error message.

3. Forgotten password: Test that a user can recover their password using the "forgotten password" feature.

4. Register new user: Test that a new user can successfully register and create an account on the website.

5. User account lockout: Test that the system can lock out a user after multiple failed login attempts.

6. User account deletion: Test that a user's account can be deleted by an administrator.

7. User account information update: Test that a user's account information can be updated successfully.

8. User account login with different devices: Test that a user can log in to their account from different devices.

9. User account login with different browsers: Test that a

In [55]:
print(response.source_nodes[0].get_content())

1.4 References
 IMDb official website httpswww.imdb.com
 REST API design principles
 OWASP security guidelines
 2. System Overview
The IMDb web platform is a threetiered architecture consisting of
Frontend A responsive web interface for user interaction.
Backend A scalable service layer handling business logic and APIs.
Database A relational and nonrelational hybrid system for storing structured and
unstructured data.
 3. Requirements
 3.1 Functional Requirements
Search Functionality Provide robust search with autosuggestions for movies, TV shows, and
Detailed Pages Display detailed information for titles, actors, and crew.
User Accounts Enable user registration, login, and profile management.
Watchlist Allow users to add titles to personal watchlists.


In [59]:
query_str = "Generate different test scenarios for IMDB page"

response = query_engine.query(query_str)

Llama.generate: 12 prefix-match hit, remaining 327 prompt tokens to eval

llama_print_timings:        load time =   11934.30 ms
llama_print_timings:      sample time =       9.57 ms /   256 runs   (    0.04 ms per token, 26758.65 tokens per second)
llama_print_timings: prompt eval time =    6993.25 ms /   327 tokens (   21.39 ms per token,    46.76 tokens per second)
llama_print_timings:        eval time =   31190.23 ms /   255 runs   (  122.31 ms per token,     8.18 tokens per second)
llama_print_timings:       total time =   38321.28 ms /   582 tokens


In [60]:
print(str(response))



Test Scenario 1: Search functionality

1. Search for a popular movie title like "Avengers: Endgame"
2. Verify that the search results display the correct information, including the movie poster, title, and summary.
3. Test autosuggestions by typing in a few characters and verifying that relevant results are displayed.

Test Scenario 2: Detailed Pages

1. Navigate to the detailed page of a popular movie like "Avengers: Endgame"
2. Verify that the page displays the correct information, including the movie poster, title, cast, crew, and user reviews.
3. Test the functionality of the "Watchlist" button and verify that it adds the movie to the user's watchlist.

Test Scenario 3: User Accounts

1. Register a new user account on IMDB.
2. Verify that the user can log in successfully and access their watchlist and profile information.
3. Test the functionality of the "Edit Profile" button and verify that it allows the user to edit their profile information.

Test Scenario 4: Watchlist

1. Add

In [61]:
print(response.source_nodes[0].get_content())

Software Design Document for IMDb Web Platform
 Version 1.0
Prepared by Your Name
Date Insert Date
Introduction
 References
System Overview
Requirements
 Functional Requirements
 NonFunctional Requirements
System Architecture
Detailed Design
 Frontend Design
 Backend Design
 Database Design
